# Imports

In [2]:
!sudo /bin/bash -c "(source /venv/bin/activate; pip install gspread_pandas)"

     |████████████████████████████████| 44 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 9.0 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 15.7 MB/s eta 0:00:01


In [5]:
!sudo /bin/bash -c "(source /venv/bin/activate; pip install google-api-python-client)"

     |████████████████████████████████| 11.5 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 12.3 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 4.7 MB/s eta 0:00:01
     |████████████████████████████████| 227 kB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 304 kB 14.3 MB/s eta 0:00:01


In [1]:
import logging
import helpers.hdbg as hdbg
import linkedin.google_api.google_file_api as google_file_api

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)


In [5]:
_LOG = logging.getLogger(__name__)
hdbg.init_logger(use_exec_path=True)

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-445be7dc-05c5-416b-8cba-add9af2959c1.json'
INFO: Saving log to file '/app/linkedin/notebooks/none.log'


# Config

# Load data

In [ ]:
# Set config files path.
c = gspread_pandas.conf.get_config("../config")

In [5]:
spreadsheet_name = "search_retired1.search_export.gsheet"
spread = gspread_pandas.Spread(spreadsheet_name, config=c)
df = spread.sheet_to_df(index=None)
print(df.shape)
df.head()

(194, 32)


,query,timestamp,error,profileUrl,fullName,firstName,lastName,companyName,title,companyId,...,vmid,linkedInProfileUrl,isPremium,isOpenLink,titleDescription,pastExperienceCompanyName,pastExperienceCompanyUrl,pastExperienceCompanyTitle,pastExperienceDate,pastExperienceDuration
0,Search5 SN,2023-07-19T18:00:39.744Z,No result found,,,,,,,,...,,,,,,,,,,
1,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.834Z,,https://www.linkedin.com/sales/lead/ACwAABgNPU...,Dr. Edgar Noumair,Dr. Edgar,Noumair,Talal and Madiha Zein AUB Innovation Park,Mentor,71418447,...,ACwAABgNPUkBxBAF3JxK0VPQmHkYRnrxXzAiHjY,https://www.linkedin.com/in/ACwAABgNPUkBxBAF3J...,FALSE,FALSE,,,,,,
2,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.834Z,,https://www.linkedin.com/sales/lead/ACwAAAHVj4...,Maureen Cusick Thomas,Maureen,Cusick Thomas,Bethesda Green,Innovation Lab Mentor,9287319,...,ACwAAAHVj44Brlumxgp-o7YP0oARKp_kKER4e6k,https://www.linkedin.com/in/ACwAAAHVj44Brlumxg...,TRUE,TRUE,The Innovation Lab Mentor Program is designed ...,,,,,
3,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.835Z,,https://www.linkedin.com/sales/lead/ACwAAAE1Yq...,Neil Davis,Neil,Davis,Innovation Works,Business Mentor,11787752,...,ACwAAAE1YqQBZoy53iiEXT0Ol-GrNh_0U0dhvqM,https://www.linkedin.com/in/ACwAAAE1YqQBZoy53i...,FALSE,FALSE,Innovation Works aims to reduce Baltimore’s ne...,,,,,
4,https://www.linkedin.com/sales/search/people?s...,2023-07-19T18:02:26.835Z,,https://www.linkedin.com/sales/lead/ACwAAACRHS...,Michael Hess,Michael,Hess,"ABS Advisory Services Group, Inc.",Chairman/CEO,,...,ACwAAACRHSQB6TCtuIxEce2hAb9lV3NM_GzNPc8,https://www.linkedin.com/in/ACwAAACRHSQB6TCtuI...,FALSE,FALSE,"Manage mergers & acquisitions, corporate and r...",,,,,


# Filter data

In [10]:
# Drop errors.
prev_len = len(df)
df = df[df["error"] == ""].reset_index(drop=True)
df = df[[col for col in df.columns if col != "error"]]
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)
df.head()

Dropped 1 rows (0.52%)


profileUrl connectionDegree  \
0  https://www.linkedin.com/in/yale-lewis-mba-049...              3rd   
1  https://www.linkedin.com/in/richard-saunders-9...              3rd   
2        https://www.linkedin.com/in/robertfarinella              3rd   
3         https://www.linkedin.com/in/scottalawrence              3rd   
4          https://www.linkedin.com/in/perrycoleman1              3rd   

                     currentJob  \
0                                 
1                                 
2  Retired at And Enjoying Life   
3                                 
4                                 

                                                 job           location  \
0                                    Retired in 2020     Winchester, VA   
1                      Retired from Federal Service.  Silver Spring, MD   
2  US Army and Civilian Hospitality/Gaming/Horse ...       Culpeper, VA   
3  Senior Manager, Software Engineering at Capita...  Silver Spring, MD   
4         IT Specialist, MBA, MSCIS, PMP, CISSP, DML      Gwynn Oak, MD   

                                                 url  \
0  https://www.linkedin.com/in/yale-lewis-mba-049...   
1  https://www.linkedin.com/in/richard-saunders-9...   
2        https://www.linkedin.com/in/robertfarinella   
3         https://www.linkedin.com/in/scottalawrence   
4          https://www.linkedin.com/in/perrycoleman1   

                                               query category  \
0  https://www.linkedin.com/search/results/people...   People   
1  https://www.linkedin.com/search/results/people...   People   
2  https://www.linkedin.com/search/results/people...   People   
3  https://www.linkedin.com/search/results/people...   People   
4  https://www.linkedin.com/search/results/people...   People   

                  timestamp               fullName firstName        lastName  \
0  2023-07-13T15:51:51.384Z        Yale Lewis, MBA      Yale      Lewis, MBA   
1  2023-07-13T15:51:51.384Z       Richard Saunders   Richard        Saunders   
2  2023-07-13T15:51:51.385Z  Robert Farinella, MBA    Robert  Farinella, MBA   
3  2023-07-13T15:51:51.385Z               Scott L.     Scott              L.   
4  2023-07-13T15:51:51.385Z        Perry M Coleman     Perry       M Coleman   

                                     profileImageUrl  \
0  https://media.licdn.com/dms/image/C4D03AQGGXPi...   
1  https://media.licdn.com/dms/image/D4E03AQHxFXh...   
2  https://media.licdn.com/dms/image/C4E03AQHPttO...   
3  https://media.licdn.com/dms/image/C4E03AQHI9E9...   
4  https://media.licdn.com/dms/image/C4E03AQEJYFZ...   

                                             summary                   name  \
0  ...Administration (MBA)  Specialties:  Financi...        Yale Lewis, MBA   
1                                                          Richard Saunders   
2                                                     Robert Farinella, MBA   
3                                                                  Scott L.   
4                                                           Perry M Coleman   

                                      additionalInfo  \
0                                                      
1                                                      
2                                                      
3  Volunteer Experience: Instructor at Adventist ...   
4                                                      

                                             pastJob sharedConnections  
0                                                                       
1                                                                       
2                                                                       
3                                                                       
4  Unmanned Aircraft System Operator at US Army -...

In [6]:
# Filter by location.
# The location you want to keep.
location = "Maryland"
prev_len = len(df)
df = df[df["location"].apply(lambda x: location in x)].reset_index(drop=True)
print(
    f"Dropped {prev_len - len(df)} rows ({round((prev_len - len(df))*100/prev_len, 2)}%)"
)
df[:10]

Dropped 10 rows (100.0%)


,profileUrl,connectionDegree,currentJob,job,location,url,query,category,timestamp,fullName,firstName,lastName,profileImageUrl,summary,name,additionalInfo,pastJob,sharedConnections


In [11]:
for col in df.columns:
    print(col)
    print(df.iloc[0][col])
    print()

profileUrl
https://www.linkedin.com/in/yale-lewis-mba-049a0922

connectionDegree
3rd

currentJob


job
Retired in 2020

location
Winchester, VA

url
https://www.linkedin.com/in/yale-lewis-mba-049a0922

query
https://www.linkedin.com/search/results/people/?contactInterest=%5B%22proBono%22%5D&geoUrn=%5B%2290000097%22%5D&keywords=Retired%20mba&origin=FACETED_SEARCH&schoolFilter=%5B%223638%22%5D&sid=m7r

category
People

timestamp
2023-07-13T15:51:51.384Z

fullName
Yale Lewis, MBA

firstName
Yale

lastName
Lewis, MBA

profileImageUrl
https://media.licdn.com/dms/image/C4D03AQGGXPiY2Gy1Sw/profile-displayphoto-shrink_100_100/0/1516993465324?e=1694649600&v=beta&t=4nf-i7u5vxcVrXvNZRaGB-MuuzePNlhf_uTPoS0xiB8

summary
...Administration (MBA)  Specialties:  Financial Planning...

name
Yale Lewis, MBA

additionalInfo


pastJob


sharedConnections




In [9]:
# Make sure profileUrl in the first column.
cols = df.columns.tolist()
if cols[0] != 'profileUrl':
    cols.remove('profileUrl')  # Remove the "profileURL" from its current position.
    cols = ['profileUrl'] + cols  # Add "profileURL" at the first column.
    df = df[cols] 
df.head()

profileUrl  \
0  https://www.linkedin.com/sales/lead/ACwAAAE1Yq...   
1  https://www.linkedin.com/sales/lead/ACwAAACRHS...   
2  https://www.linkedin.com/sales/lead/ACwAAAAQQk...   
3  https://www.linkedin.com/sales/lead/ACwAAADeWj...   
4  https://www.linkedin.com/sales/lead/ACwAAAAY9Y...   

                                               query  \
0  https://www.linkedin.com/sales/search/people?s...   
1  https://www.linkedin.com/sales/search/people?s...   
2  https://www.linkedin.com/sales/search/people?s...   
3  https://www.linkedin.com/sales/search/people?s...   
4  https://www.linkedin.com/sales/search/people?s...   

                  timestamp          fullName firstName   lastName  \
0  2023-07-19T18:02:26.835Z        Neil Davis      Neil      Davis   
1  2023-07-19T18:02:26.835Z      Michael Hess   Michael       Hess   
2  2023-07-19T18:02:26.835Z  Barrie Zucal, MS    Barrie  Zucal, MS   
3  2023-07-19T18:02:26.835Z      Leslie Traub    Leslie      Traub   
4  2023-07-19T18:02:26.836Z   Theresa Daytner   Theresa    Daytner   

                         companyName  \
0                   Innovation Works   
1  ABS Advisory Services Group, Inc.   
2                   Zucal Consulting   
3           Inclusion Allies Network   
4              Conscious Venture Lab   

                                               title companyId  \
0                                    Business Mentor  11787752   
1                                       Chairman/CEO             
2              Independent Mentor, Coach, Consultant             
3                              Co-Founder and Mentor             
4  Lead Mentor | Angel Investor | Executive Coach...   3356498   

                                        companyUrl  ...  \
0  https://www.linkedin.com/sales/company/11787752  ...   
1                                                   ...   
2                                                   ...   
3                                                   ...   
4   https://www.linkedin.com/sales/company/3356498  ...   

                                      vmid  \
0  ACwAAAE1YqQBZoy53iiEXT0Ol-GrNh_0U0dhvqM   
1  ACwAAACRHSQB6TCtuIxEce2hAb9lV3NM_GzNPc8   
2  ACwAAAAQQkoBFSrQYiDk3QVKcRprtPvRSXe7ajk   
3  ACwAAADeWjEBRuyhZkIF7X7fpy6uBoxiXLQWqLY   
4  ACwAAAAY9YQBq8q_bnRNDw_10z49i6IR2gD7RKQ   

                                  linkedInProfileUrl isPremium isOpenLink  \
0  https://www.linkedin.com/in/ACwAAAE1YqQBZoy53i...     FALSE      FALSE   
1  https://www.linkedin.com/in/ACwAAACRHSQB6TCtuI...     FALSE      FALSE   
2  https://www.linkedin.com/in/ACwAAAAQQkoBFSrQYi...     FALSE      FALSE   
3  https://www.linkedin.com/in/ACwAAADeWjEBRuyhZk...     FALSE      FALSE   
4  https://www.linkedin.com/in/ACwAAAAY9YQBq8q_bn...      TRUE      FALSE   

                                    titleDescription  \
0  Innovation Works aims to reduce Baltimore’s ne...   
1  Manage mergers & acquisitions, corporate and r...   
2  I work with talented women in global roles who...   
3  A coalition of organizations  and practitioner...   
4  As a Lead Mentor for the Conscious Venture Lab...   

  pastExperienceCompanyName pastExperienceCompanyUrl  \
0                                                      
1                                                      
2                                                      
3                                                      
4                                                      

  pastExperienceCompanyTitle pastExperienceDate pastExperienceDuration  
0                                                                       
1                                                                       
2                                                                       
3                                                                       
4                                                                       

[5 rows x 31 columns]

In [10]:
print(df.shape)

(82, 31)


# Save filtered data to gsheet

## Create a empty Google sheet

In [20]:
import logging
import pandas as pd
import helpers.hdbg as hdbg
import linkedin.google_api.google_file_api as google_file_api

In [2]:
_LOG = logging.getLogger(__name__)
hdbg.init_logger(use_exec_path=True)

INFO: > cmd='/venv/lib/python3.8/site-packages/ipykernel_launcher.py -f /home/.local/share/jupyter/runtime/kernel-4dce2228-b09e-461c-ae34-9166cd52b6a6.json'
INFO: Saving log to file '/app/linkedin/notebooks/none.log'


In [3]:
gapi = google_file_api.GoogleFileApi()

INFO  file_cache is only supported with oauth2client<4.0.0


In [ ]:
gfolder_id = ''

In [17]:
name = 'SN_Search5_Yiyun'
# gdrive_folder : dict, the id and the name of the Google Drive folder.
gdrive_folder  = gapi.get_folder_id_by_name(name)

INFO  Found folder: {'id': '1XWNGDnJrVICHAe-6V2cnoSklZpk0APc_', 'name': 'SN_Search5_Yiyun'}


In [25]:
# if you want to use another folder id, please change the folder id manually.
# gdrive_folder  = {'id': '1XWNGDnJrVICHAe-6V2cnoSklZpk0APc_', 'name': 'SN_Search5_Yiyun'} 

In [22]:
spreadsheets_in_folder = gapi.get_spreadsheets_in_gdirve_folder(gdrive_folder.get("id"))
df = pd.DataFrame(spreadsheets_in_folder)
df

,id,name
0,1j_kD3o5ZQ4CPDwSM5xPeALF79DpRBwm1C99Djxcn1es,sn_search5.profile_export.filtered.gsheet
1,1pRfJcgr7kHXUt_jpwWr460IQHcx8gqzEV4vLCfSX9F4,sn_search5.search_export.filtered.gsheet
2,1cqtT_FUIwbhfIH7_Vh7JDCFC83z7UlWDlj5dXdpOT5I,sn_search5.profile_export.gsheet
3,1XJtEwn_Nu46xpQDCpJaqTHs0auY2CeOgDDJkF0s52NQ,sn_search5.search_export.gsheet


In [ ]:
gsheet_name = 'sn_search5.search_export.filtered.gsheet'

## Create a new Google Sheet

In [27]:
"""
Create a new Google file (sheet or doc).

:param gfile_type: str, the type of the Google file ('sheet' or 'doc').
:param gfile_name: str, the name of the new Google file.
:param folder_id: str, the id of the Google Drive folder.
:param user: str, the email address of the user to share the Google file (Optional).
:return: None
"""
gfile_type = 'sheet'
user = ''

In [29]:
gapi.create_empty_google_file(
    gfile_type = gfile_type,
    gfile_name = gsheet_name,
    gfolder_id = gfolder_id,
    gdrive_folder = gdrive_folder,
    user = user
)

INFO  file_cache is only supported with oauth2client<4.0.0
INFO  Created a new Google sheet 'sn_search5.search_export.filtered.gsheet'.
INFO  Move the new Google sheet 'sn_search5.search_export.filtered.gsheet' to the dir 'SN_Search5_Yiyun'
INFO  Finished creating the new Google sheet 'sn_search5.search_export.filtered.gsheet'.


## Save filtered data

In [30]:
# A Google sheet with this name should already exist on the drive.
spread2 = gspread_pandas.Spread(
    gsheet_name,
    sheet="Sheet1",
    create_sheet=True,
)
spread2.df_to_sheet(df, index=False)